In [ ]:
import cv2
import numpy as np
import random
import os
import csv
from tensorflow.keras.models import load_model

In [ ]:
def extract_random_frames(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if total_frames < num_frames:
        raise ValueError(f"The video {video_path} doesn't have enough frames.")
    
    frame_indices = random.sample(range(total_frames), num_frames)
    frames = []

    for frame_idx in sorted(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            print(f"Failed to read frame at index {frame_idx} in video {video_path}")
    
    cap.release()
    return frames

In [ ]:
def predict_frame(model, frame):
    # Preprocess the frame if necessary (e.g., resizing, normalization)
    processed_frame = cv2.resize(frame, (model.input_shape[1], model.input_shape[2]))
    processed_frame = processed_frame / 255.0
    processed_frame = np.expand_dims(processed_frame, axis=0)
    
    # Predict the class (0 or 1)
    prediction = model.predict(processed_frame)
    return np.argmax(prediction, axis=1)[0]

In [ ]:
def process_video(video_path, models):
    frames = extract_random_frames(video_path, num_frames=100)
    predictions = {model_name: [] for model_name in models.keys()}

    for frame in frames:
        for model_name, model in models.items():
            prediction = predict_frame(model, frame)
            predictions[model_name].append(prediction)
    
    average_predictions = {model_name: np.mean(preds) for model_name, preds in predictions.items()}
    return average_predictions

In [ ]:
def main(dataset_path, model_paths, output_csv):
    models = {model_name: load_model(model_path) for model_name, model_path in model_paths.items()}
    
    # Read video paths from dataset.txt
    with open(dataset_path, 'r') as file:
        videos = [line.strip() for line in file.readlines()]

    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ['Video'] + list(models.keys())
        writer.writerow(header)

        for video in videos:
            if not os.path.exists(video):
                print(f"Video file {video} does not exist. Skipping.")
                continue

            average_predictions = process_video(video, models)
            row = [os.path.basename(video)] + [average_predictions[model_name] for model_name in models.keys()]
            writer.writerow(row)

In [ ]:
dataset_path = 'dataset.txt'
model_paths = {
    'model1': 'path_to_model1.h5',
    'model2': 'path_to_model2.h5',
    'model3': 'path_to_model3.h5'
}
output_csv = 'predictions.csv'

main(dataset_path, model_paths, output_csv)